In [ ]:
#@title Mount Google Drive
from google.colab import drive

!nvidia-smi
# !nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

drive.mount('/content/gdrive')

In [ ]:
from IPython.display import clear_output

#@title Install Radiata and Requirements
#@markdown Radiata Install options
GoogleDrive_Install = True #@param {type:"boolean"}

#@markdown Requirements options
Torch_nightly = False #@param {type:"boolean"}
xFormers = True #@param {type:"boolean"}
TensorRT = True #@param {type:"boolean"}

if GoogleDrive_Install:
  %cd /content/gdrive/MyDrive
else:
  %cd /content/

!git clone https://github.com/ddPn08/Radiata.git

%cd Radiata
if Torch_nightly:
  !pip install --pre torch torchvision --force-reinstall \
    --index-url https://download.pytorch.org/whl/nightly/cu118
  clear_output()

!pip install -q -r requirements/base.txt
clear_output()

if TensorRT:
  !pip install -q -r requirements/tensorrt.txt
  !pip install tensorrt
  clear_output()

if xFormers:
  !pip install -U xformers
  clear_output()
  !python -m xformers.info

print("🔄 Requirements Installed!")

In [ ]:
#@title Launch Radiata
#@markdown Radiata Dictionary
Radiata_Dir = '/content/gdrive/MyDrive/Radiata' #@param {type:"string"}

%cd $Radiata_Dir

#@markdown Radiata Version Control
Operation = "Update" #@param ["Update", "Rollback", "Skip"]
Commit_SHA = '' #@param {type:"string"}

if Operation == "Rollback":
  %rm -f $Radiata_Dir/.git/index.lock
  !git config core.filemode false
  !git reset --hard $Commit_SHA
  !git pull
elif Operation == "Update":
  %rm -f $Radiata_Dir/.git/index.lock
  !git config core.filemode false
  !git reset --hard main
  !git pull

#@markdown Radiata Launch Options
xFormers = True #@param {type:"boolean"}
TensorRT = True #@param {type:"boolean"}
Deepfloyd_IF = False #@param {type:"boolean"}
Launch_Mode = "inline" #@param ["inline", "gradio", "ngrok"]
ngrokToken = "" #@param {type:"string"}

cmd_args = "--skip-install "
cmd_args += "--xformers " if xFormers
cmd_args += "--tensorrt " if TensorRT
cmd_args += "--deepfloyd_if " if Deepfloyd_IF


def inline_launch():
  import modules.ui as ui
  import webui

  webui.pre_load()
  demo = ui.create_ui().queue(64)

  demo = demo.launch(
    inline=True,
    share=True,
    prevent_thread_lock=True,
    debug=True
  )

def gradio_launch():
  cmd_args += "--share"
  !COMMANDLINE_ARGS="{cmd_args}" python launch.py

def ngrok_launch():
  try:
    from pyngrok import conf, ngrok
  except:
    from pyngrok import conf, ngrok
    conf.get_default().auth_token = ngrokToken
    conf.get_default().monitor_thread = False
    ssh_tunnels = ngrok.get_tunnels(conf.get_default())
    if len(ssh_tunnels) == 0:
        ssh_tunnel = ngrok.connect(7860)
        print('address：'+ssh_tunnel.public_url)
    else:
        print('address：'+ssh_tunnels[0].public_url)
  cmd_args += "--share"
  !COMMANDLINE_ARGS="{cmd_args}" python launch.py

import os
os.environ["COMMANDLINE_ARGS"] = cmd_args
os.environ["CUDA_MODULE_LOADING"] = "LAZY"

if Launch_Mode == "inline":
  inline_launch()
elif Launch_Mode == "gradio":
  gradio_launch()
elif Launch_Mode == "ngrok":
  ngrok_launch()